In [2]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import sys
from cycifsuite.plate_based_analysis import per_well_analysis
from cycifsuite.get_data import read_synapse_file
from cycifsuite.combat import combat
%load_ext autoreload
%autoreload 2

In [4]:
path = 'N:/HiTS Projects and Data/Personal/Jake/mcf10a'
os.chdir(path)
pooled_expr_data = pd.read_hdf('intensity_nuc_cytoplasm.hdf')
pooled_metadata = pd.read_csv('proper_FFC_metadata.csv',index_col=0)
pooled_metadata = pooled_metadata[(pooled_metadata.labeled_as_lost=='No')&(pooled_metadata.num_nuclei_in_mask==1)]
pooled_expr_data = pooled_expr_data.reindex(pooled_metadata.index).dropna(axis=1)
pooled_expr_data = combat(pooled_expr_data.transpose(), pooled_metadata.replicate).transpose()
pooled_expr_data = pooled_expr_data[sorted(pooled_expr_data.columns)]

found 3 batches
found 0 numerical covariates...
found 0 categorical variables:	
Standardizing Data across genes.
Fitting L/S model and finding priors
Finding parametric adjustments


Adjusting data


# PBS as control
## PBS wells on each plate were used as control

In [6]:
control_name = 'PBS'
output_prefix = ''
output_folder = 'well_based_analysis/pbs_as_control'
pwa = per_well_analysis(pooled_expr_data,pooled_metadata[pooled_metadata.ligand!='ctrl'],batch_col='Plate',
                        drug_col='ligand',dose_col='dose',time_col='time',output_path=output_folder, control_name = control_name, output_prefix=output_prefix)
pwa.make_contrast(batch_wise=True)
pwa.descriptive_analysis()
pwa.contrast_based_distance()
pwa.contrast_based_differential_analysis()
pwa.plot_dist_violinplot('ligand','time')
pwa.make_fc_heamtap_matrix(['ligand','time','Batch'],output_fname=output_prefix)
pwa.make_heatmaps(style='simple')
pwa.make_dotted_heatmap()
pwa.make_swarm_plot()

# T0 as control
## T0 of each replicate were used as control, this will require running bach normalization

In [7]:
control_name = 'ctrl'
output_prefix = ''
output_folder = 'well_based_analysis/t0_as_control'
pwa = per_well_analysis(pooled_expr_data,pooled_metadata,batch_col='replicate',output_path=output_folder, 
                        drug_col='ligand',dose_col='dose',time_col='time',control_name = control_name, output_prefix=output_prefix)
pwa.make_contrast(batch_wise=True)
pwa.descriptive_analysis()
pwa.contrast_based_distance()
pwa.contrast_based_differential_analysis()
pwa.plot_dist_violinplot('ligand','time')
pwa.make_fc_heamtap_matrix(['ligand','time','Batch'],output_fname=output_prefix)
pwa.make_heatmaps(style='simple')
pwa.make_dotted_heatmap()
pwa.make_swarm_plot()

## EGF_1h as control

In [8]:
pooled_metadata.loc[pooled_metadata.condition=='EGF_1','ligand'] = 'EGF_1'
control_name = 'EGF_1'
output_prefix = ''
output_folder = 'well_based_analysis/egf_as_control'
pwa = per_well_analysis(pooled_expr_data,pooled_metadata[pooled_metadata.ligand!='ctrl'],batch_col='replicate',output_path=output_folder,
                        drug_col='ligand',dose_col='dose',time_col='time',control_name = control_name, output_prefix=output_prefix)
pwa.make_contrast(batch_wise=True)
# pwa.descriptive_analysis()
pwa.contrast_based_distance()
pwa.contrast_based_differential_analysis()
pwa.plot_dist_violinplot('ligand','time')
pwa.make_fc_heamtap_matrix(['ligand','time','Batch'],output_fname=output_prefix)
pwa.make_heatmaps(style='simple')
pwa.make_dotted_heatmap()
pwa.make_swarm_plot()